# We will be building a Coroutine pipeline for Word Counter

In [10]:
import re 

# Producer 
def word_count_producer(text, sanitize_coroutine):
    for word in text.split(' '):
        sanitize_coroutine.send(word)
    sanitize_coroutine.close()


# Filter1: Remove Punctuation
def remove_punctuation(next_coroutine):
    print("Start punctuation removal")
    try:
        while True:
            word = yield
            next_coroutine.send(re.sub(r'[^\w\s]','',word))
    except:
        next_coroutine.close()

# Filter2: changes the case to lowercase
def sanitize_word(next_coroutine):
    print("Start lowering words")
    try:
        while True:
            word = yield
            next_coroutine.send(word.lower())
    except:
        next_coroutine.close()
        
        
# Sink: Prints the word_count
def print_word_count():
    word_count = {}
    try:
        while True:
            word = yield
            if word_count.get(word, None):
                word_count[word]+=1
            else:
                word_count[word] = 1
    except GeneratorExit:
        print(word_count)
        

In [11]:
#First initialize the sink, and then continue in reverse order
my_wc = print_word_count()


In [12]:
# Call next over sink
next(my_wc)

In [13]:
# Initialize the filters in reverse and 
# chain the sink to the last filter
sn_words = sanitize_word(my_wc)

In [14]:
next(sn_words)

Start lowering words


In [15]:
# Similarly, initialize the filters in reverse order
# and attach the previously initialized filter to them
rem_words = remove_punctuation(sn_words)

{}


In [16]:
next(rem_words)

Start punctuation removal


In [17]:
my_str = "hello this is a string. This is a good String."

In [18]:
# Finally call the producer with the last initialized filter
word_count_producer(my_str,rem_words)

{'hello': 1, 'this': 2, 'is': 2, 'a': 2, 'string': 2, 'good': 1}
